***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-5660  GCE       4                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Feb 29 06:09 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '318305570_207757535_bucket'
full_path = f"gs://{bucket_name}/"
paths=[]
import glob
bin_files = glob.glob(full_path + "/*.bin")
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
        if not ((b.name.startswith('index_posting_locs_title/') or b.name.startswith('index_posting_locs_training/') or b.name.startswith('index_posting_locs_body/')) and (b.name.endswith('.bin') or b.name.endswith('.pickle') or b.name.endswith('.pkl')) or b.name == 'graphframes.sh'):
            paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [9]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd
filtered_doc_text_pairs = doc_text_pairs.filter(lambda x: "genetics" in x[0])

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [10]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [11]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_body_gcp_training.py

inverted_index_body_gcp_training.py


In [12]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_body_gcp_training.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [13]:
from inverted_index_body_gcp_training import *

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes:
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab.

In [14]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

def stemming(tokens):
    stemmer = PorterStemmer()
    list_of_tokens = [stemmer.stem(x) for x in tokens]
    return list_of_tokens

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in
    `all_stopwords` and return entries that will go into our posting lists.
    Parameters:
    -----------
    text: str
      Text of one document
    id: int
      Document id
    Returns:
    --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
    '''
    word_freq = {}
    for token in text:
      if token not in all_stopwords:
          if token in word_freq:
              word_freq[token] += 1
          else:
              word_freq[token] = 1
    return [(token, (id, freq)) for token, freq in word_freq.items()]

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # Sort the unsorted posting list by wiki_id
  sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])
  return sorted_pl

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # Count the number of documents each token appears in
  token_df = postings.map(lambda x: (x[0], len(x[1]))).distinct()
  return token_df

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  # Partition postings into buckets
  bucketed_postings = postings.groupBy(lambda x: token2bucket_id(x[0]))

  # Write posting lists to disk and get posting locations for each bucket
  posting_locations = bucketed_postings.map(lambda x: InvertedIndex.write_a_posting_list((x[0], list(x[1])),'index_posting_locs_training', bucket_name))

  return posting_locations

In [17]:
tokens_doc_pairs = filtered_doc_text_pairs.map(lambda x: ([token.group() for token in RE_WORD.finditer(x[0].lower())], x[1]))
tokens_doc_pairs_stemmed = tokens_doc_pairs.map(lambda x: (stemming(x[0]), x[1]))
word_counts = tokens_doc_pairs_stemmed.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
postings_filtered = postings.filter(lambda x: len(x[1])>10)
filtered_rdd = postings_filtered.flatMap(lambda x: [(item[0], (x[0], item[1])) for item in x[1]]) \
                      .groupByKey() \
                      .map(lambda x: (x[0], list(x[1])))
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
posting_locs_list = partition_postings_and_write(postings_filtered).collect()

In [15]:
# test index construction time
print(postings_filtered.take(2))
print(filtered_rdd.take(2))

[('nomenclature', [(2494, 1), (2965, 2), (3974, 1), (4183, 5), (4493, 1), (5180, 4), (5376, 4), (5876, 1), (6438, 1), (6682, 7), (6896, 2), (7626, 1), (8311, 1), (8498, 8), (10253, 2), (12339, 1), (12796, 1), (14029, 1), (17646, 1), (18393, 2), (19053, 1), (19702, 4), (21265, 1), (22984, 1), (23033, 1), (23749, 1), (23924, 2), (23962, 1), (24354, 1), (24454, 2), (26700, 1), (27067, 1), (27813, 1), (28639, 1), (29252, 1), (30463, 13), (31783, 1), (32653, 1), (33702, 1), (34413, 4), (38310, 1), (38890, 1), (39389, 1), (40114, 2), (41244, 1), (42967, 1), (45609, 1), (52924, 1), (53130, 1), (53664, 3), (53948, 1), (54025, 1), (56276, 3), (58708, 1), (59018, 2), (61593, 1), (61762, 1), (61763, 6), (62290, 2), (62696, 2), (63790, 1), (66512, 1), (67168, 1), (68300, 3), (88595, 2), (89792, 1), (89896, 1), (99645, 1), (149544, 1), (161535, 1), (166751, 1), (173204, 1), (186367, 1), (206301, 1), (208458, 1), (229116, 2), (243749, 1), (277102, 1), (292297, 1), (301737, 1), (302013, 3), (331755, 

[(31372, [('works', 3), ('humans', 4), ('emeritus', 1), ('1924', 2), ('entomologists', 3), ('direct', 1), ('failure', 1), ('problems', 1), ('differing', 1), ('orthodox', 4), ('completely', 1), ('toxic', 1), ('early', 1), ('spreading', 1), ('favored', 1), ('thrombosis', 1), ('1972', 1), ('faculty', 2), ('great', 2), ('whether', 1), ('francisco', 3), ('gone', 1), ('city', 1), ('date', 1), ('simply', 1), ('validity', 1), ('editions', 3), ('priest', 1), ('2nd', 1), ('chronic', 1), ('outstanding', 1), ('eliminated', 1), ('advancements', 1), ('helped', 1), ('ethnic', 1), ('talks', 1), ('receive', 1), ('heredity', 4), ('infertile', 1), ('microevolution', 1), ('thomas', 1), ('best', 1), ('canada', 1), ('1921', 1), ('social', 1), ('remove', 1), ('attended', 1), ('middle-aged', 1), ('central', 1), ('identical', 1), ('adaptation', 1), ('ernst', 1), ('raw', 1), ('franklin', 1), ('opportunity', 1), ('italy', 1), ('public', 1), ('biologists', 2), ('morphological', 1), ('1977', 1), ('american', 10), 

In [18]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='index_posting_locs_training'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)
#             print(len(super_posting_locs))
print(super_posting_locs.get('genetics'))

None


Putting it all together

In [19]:
import math
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
#partition_func
inverted.posting_locs = super_posting_locs
inverted.doc_count = filtered_rdd.count()
N = inverted.doc_count
# inverted._posting_list = dict(postings_filtered.collect())
##############################################################
##### need to change filtered_doc_text_pairs to doc_text_pairs
##### when testing the whole corpus
##############################################################
# inverted.doc_count = filtered_doc_text_pairs.count()
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# Iterate through tokens_doc_pairs to compute TF-IDF for each token in each document
sum_len_of_all_docs = 0
for doc_id, words_in_doc in filtered_rdd.take(N):
  sum_len_docs = 0
  sum_doc_tf_idf = 0
  for token, tf in words_in_doc:
    if inverted.df.get(token) is not None:
      df = inverted.df[token]
      idf = math.log(inverted.doc_count / df, 10)
      sum_len_docs += tf
      sum_doc_tf_idf += math.pow(tf*idf, 2)
  inverted.doc_tf_idf_doc_len[doc_id] = (math.sqrt(sum_doc_tf_idf), sum_len_docs)
  sum_len_of_all_docs += sum_len_docs
inverted.doc_len_avg = sum_len_of_all_docs / inverted.doc_count
inverted.write_index('.', 'index_body_genetics')
# upload to gs
index_src = "index_body_genetics.pkl"
index_dst = f'gs://{bucket_name}/index_posting_locs_training/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index_body_genetics.pkl [Content-Type=application/octet-stream]...
/ [1 files][  1.4 MiB/  1.4 MiB]                                                
Operation completed over 1 objects/1.4 MiB.                                      


In [17]:
# print(inverted.posting_locs.get('friends'))
# print(N)

[('105_000.bin', 58962)]
12905


In [20]:
!gsutil ls -lh $index_dst

  1.39 MiB  2024-02-29T12:28:34Z  gs://318305570_207757535_bucket/index_posting_locs_training/index_body_genetics.pkl
TOTAL: 1 objects, 1456200 bytes (1.39 MiB)


In [31]:
temp_index = InvertedIndex.read_index('index_posting_locs_training', 'index_body_genetics',bucket_name)
print(temp_index.posting_locs.get('genet'))
post = temp_index.read_a_posting_list('.', 'genet', bucket_name)
print(post)

[('index_posting_locs_training/5_000.bin', 201720)]
[(25, 26), (569, 6), (627, 20), (698, 1), (777, 1), (786, 2), (852, 3), (874, 4), (1051, 1), (1910, 4), (1911, 10), (1913, 1), (1914, 5), (1916, 4), (2154, 9), (2472, 7), (2494, 10), (2594, 6), (2766, 1), (2909, 7), (2965, 11), (3029, 11), (3034, 1), (3209, 9), (3347, 1), (3416, 2), (3717, 6), (3878, 26), (3954, 11), (3965, 2), (3974, 1), (3978, 1), (4011, 1), (4031, 1), (4183, 24), (4185, 7), (4213, 5), (4214, 8), (4292, 7), (4294, 3), (4402, 9), (4417, 1), (4493, 19), (4502, 44), (4583, 11), (4620, 2), (4805, 1), (4868, 3), (5009, 26), (5180, 1), (5221, 2), (5321, 1), (5326, 3), (5331, 1), (5376, 3), (5636, 1), (5693, 2), (5717, 2), (5823, 2), (5836, 1), (5876, 6), (6013, 1), (6080, 1), (6339, 4), (6438, 18), (6546, 13), (6710, 11), (6736, 1), (6828, 1), (6868, 1), (6896, 2), (6910, 18), (6933, 1), (7080, 1), (7172, 7), (7316, 8), (7329, 1), (7330, 4), (7333, 3), (7626, 1), (7638, 1), (7732, 5), (7844, 6), (7955, 35), (8013, 5), (81

# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
def generate_graph(pages):
    # Extract edges from pages
    edges = pages.flatMap(extract_edges).distinct()

    # Extract vertices from edges
    vertices = edges.flatMap(lambda edge: ((edge[0],), (edge[1],))).distinct()

    return edges, vertices

def extract_edges(row):
    edges = []
    for anchor_text in row[1]:
          edges.append((row[0], anchor_text[0]))
    return edges

In [ ]:
t_start = time()
pages_links = parquetFile.select ("id","anchor_text").rdd
# construct the graph
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

24/02/16 17:30:02 WARN YarnAllocator: Container from a bad node: container_1708100150783_0001_01_000012 on host: cluster-d9c8-w-1.c.bubbly-bastion-414507.internal. Exit status: 143. Diagnostics: [2024-02-16 17:30:01.385]Container killed on request. Exit code is 143
[2024-02-16 17:30:01.386]Container exited with a non-zero exit code 143. 
[2024-02-16 17:30:01.386]Killed by external signal
.
24/02/16 17:30:02 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 12 for reason Container from a bad node: container_1708100150783_0001_01_000012 on host: cluster-d9c8-w-1.c.bubbly-bastion-414507.internal. Exit status: 143. Diagnostics: [2024-02-16 17:30:01.385]Container killed on request. Exit code is 143
[2024-02-16 17:30:01.386]Container exited with a non-zero exit code 143. 
[2024-02-16 17:30:01.386]Killed by external signal
.
24/02/16 17:30:02 ERROR YarnScheduler: Lost executor 12 on cluster-d9c8-w-1.c.bubbly-bastion-414507.internal: Container from a bad nod

+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750|  9913.72878216077|
|  10568| 5385.349263642037|
|  32927| 5282.081575765277|
|  30680| 5128.233709604119|
|5843419| 4957.567686263868|
|  68253| 4769.278265355163|
|  31717|  4486.35018054831|
|  11867|4146.4146509127695|
|  14533| 3996.466440885503|
| 645042|3531.6270898037437|
|  17867|3246.0983906041406|
|5042916| 2991.945739166176|
|4689264|2982.3248830417474|
|  14532|2934.7468292031704|
|  25391|2903.5462235133978|
|   5405|2891.4163291546356|
|4764461| 2834.366987332661|
|  15573|  2783.86511815884|
|   9316|  2782.03964641377|
|8569916|2775.2861918400167|
+-------+------------------+
only showing top 20 rows



In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*120

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete
the reporting requirements for assignment #3.

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

14.28 GiB    gs://wikidata_preprocessed


In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

5.92 GiB     gs://318305570_netta/postings_gcp


In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 0.011
print(f'I used {cost} USD credit during the course of this assignment')

I used 0.011 USD credit during the course of this assignment


**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour.


In [ ]:
#If you have decided to do the bonus task - please copy the code here

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag